# Analyse Geschlecht

In [2]:
# Bibliotheken laden
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import seaborn as sns
from shapely import wkt
from geopy.distance import geodesic
import numpy as np
import geopandas as gpd
from folium import MacroElement
from jinja2 import Template
import requests
import folium
from shapely import from_wkt
import contextily as ctx

In [3]:
# CSV-Datei einlesen
file_path_persons = '../../data/clean/Data_Masterarbeit - list_named_after_humans.csv'
file_path_all_streets = '../../data/clean/Data_Masterarbeit - list_complete.csv'
file_path_street_layout = "../../data/raw/lhd_strassenzuege.csv"

df_persons = pd.read_csv(file_path_persons, sep=",")
df_all = pd.read_csv(file_path_all_streets, sep=",")
df_streets = pd.read_csv(file_path_street_layout, sep=";")

## Verteilung Diagramm

In [8]:
# Schritt 1: Kategorie bestimmen
category = "Geschlecht"

# 1. Berechne die Gesamtanzahl für die Prozentrechnung
category_counter = df_persons["Geschlecht"].value_counts()
category_counter = category_counter.sort_values(ascending=False)

total = category_counter.sum()

# 2. Erstelle die benutzerdefinierten Labels
custom_labels = []
for count in category_counter.values:
    pct = (count / total) * 100
    # Die absolute Zahl ist der 'count' selbst
    label_string = f"{pct:.1f}%\n({count})"
    custom_labels.append(label_string)

# 3. Erstelle eine Figure und eine Axes
fig, ax = plt.subplots()

# 3.1  Farbzuweisung
colors_for_plot = []
# Gehe durch die Indizes (Kategorien) in category_counter
for item in category_counter.index:
    if item == "weiblich":
        colors_for_plot.append("red") # Rot für "weiblich"
    elif item == "männlich":
        colors_for_plot.append("blue") # Blau für "männlich"
    else:
        colors_for_plot.append("gray") # Grau für alle anderen
# 4. Erstelle das Balkendiagramm auf der 'ax'
bars = ax.barh(
    category_counter.index,
    category_counter.values,
    color=colors_for_plot
)

# 5. Füge die benutzerdefinierten Labels über den Balken hinzuent
ax.bar_label(bars, labels=custom_labels)

# 6. Setze den Titel
ax.set_title(f"Verteilung nach {category} (nach Personen benannte Straßen)")

# 7. Die Labels rechts nicht abschneiden
ax.margins(x=0.15)

ax.invert_yaxis()

# 9. Diagramm speichern
fig.savefig(f"../../results/charts/bar_charts/Verteilung_{category}.png", bbox_inches="tight")

plt.close()


## Distanzanalyse

In [ ]:
# WKT in Geometrie umwandeln
def safe_wkt(val):
    if pd.isna(val) or str(val).strip() == "":
        return None
    try:
        return wkt.loads(str(val).replace("Point", "POINT"))
    except:
        return None

df_all["geometry"] = df_all["Koordinaten [WD]"].apply(safe_wkt)
df_all["lon"] = df_all["geometry"].apply(lambda g: g.x if g else np.nan)
df_all["lat"] = df_all["geometry"].apply(lambda g: g.y if g else np.nan)

# Merge über ID
df = pd.merge(df_persons, df_all, on="ID", how="inner")

# Altmarkt Dresden als Zentrum
center = (51.0504, 13.7373)

# Distanz berechnen
df["distanz_km"] = np.nan
mask = df["lat"].notna() & df["lon"].notna()
df.loc[mask, "distanz_km"] = df.loc[mask].apply(
    lambda row: geodesic((row["lat"], row["lon"]), center).kilometers,
    axis=1
)

# Reihenfolge der Kategorien (wie im Boxplot)
order = df["Geschlecht"].dropna().unique()

# Statistische Kennzahlen je Geschlecht
stats = df.groupby("Geschlecht")["distanz_km"].agg(["mean", "median", "count"]).reindex(order)

custom_palette = {
    "weiblich": "red",
    "männlich": "blue",
    "trans* Frau": "gray"
}

# Boxplot zeichnen
plt.figure(figsize=(8,6))
sns.boxplot(
    data=df[df["distanz_km"].notna()],
    x="Geschlecht",
    y="distanz_km",
    palette=custom_palette,
    order=order
)

plt.title("Distanz von nach Männern/Frauen benannten Straßen zum Altmarkt (Dresden)")
plt.xlabel("Geschlecht")
plt.ylabel("Distanz zum Zentrum (km)")

# Mittelwert & Median über jeder Box eintragen
for i, (geschlecht, row) in enumerate(stats.iterrows()):
    mean_val = row["mean"]
    median_val = row["median"]
    plt.text(
        i,
        mean_val + 0.2,
        f"Ø {mean_val:.2f} km\nMed {median_val:.2f} km",
        ha="center",
        color="black",
        fontsize=10,
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="gray")
    )

# Legende für Erklärung
plt.legend(
    [plt.Line2D([0], [0], color="white")],
    ["Ø = Mittelwert | Med = Median"],
    loc="upper right",
    frameon=False
)

# Diagramm speichern
plt.savefig("../../results/charts/boxplots/distanzanalyse_geschlechter_boxplot.png", dpi=300)
plt.close()

C:\Users\juliu\AppData\Local\Temp\ipykernel_3360\3273404696.py:42: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(


## Karte Stadtbezirke

In [ ]:
# 1. Daten laden & Geometrien bereinigen

def safe_from_wkt(x):
    """Sicheres Parsen von WKT → Shapely-Geometrie, sonst None."""
    if pd.isna(x):
        return None
    try:
        return from_wkt(str(x))
    except Exception:
        return None


# SRID entfernen + Geometrien parsen
df_streets["geometry"] = (
    df_streets["geom"]
    .astype(str)
    .str.replace(r"SRID=\d+;", "", regex=True)
    .apply(safe_from_wkt)
)

# Als GeoDataFrame mit gültigen Geometrien
df_streets = gpd.GeoDataFrame(df_streets, geometry="geometry", crs="EPSG:4326")
df_streets = df_streets.dropna(subset=["geometry"])


# Straßennamen vereinheitlichen
df_streets["strasse_clean"] = df_streets["strasse"].str.strip().str.lower()
df_persons["street_name_clean"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge Gender-Info
edges = df_streets.merge(
    df_persons[["street_name_clean", "Geschlecht"]],
    left_on="strasse_clean", right_on="street_name_clean", how="left"
)

# 3. Stadtbezirke laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L139/items"
stadtbezirke_json = requests.get(stadtbezirke_url).json()
stadtbezirke = gpd.GeoDataFrame.from_features(stadtbezirke_json["features"])
stadtbezirke = stadtbezirke.set_crs(epsg=4326)

# 4. Räumliche Verknüpfung: Straße ↔ Bezirk
edges_mit_bezirk = gpd.sjoin(edges, stadtbezirke, how="left", predicate="intersects")

# 5. NUR nach Personen benannte Straßen
personen_strassen = edges_mit_bezirk.dropna(subset=["Geschlecht"])
personen_strassen_einzigartig = personen_strassen.drop_duplicates(subset=["bez", "strasse_clean"])

# Anteil weiblich je Bezirk
gesamt = personen_strassen_einzigartig.groupby("bez").size()
weiblich = (
    personen_strassen_einzigartig[personen_strassen_einzigartig["Geschlecht"] == "weiblich"]
    .groupby("bez")
    .size()
)
anteile = (weiblich / gesamt).fillna(0).reset_index()
anteile.columns = ["bez", "anteil_weiblich"]

stadtbezirke = stadtbezirke.merge(anteile, on="bez", how="left")
stadtbezirke["anteil_weiblich"] = stadtbezirke["anteil_weiblich"].fillna(0)

# 6. Karte erstellen
center = stadtbezirke.geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=12)

# Choropleth
folium.Choropleth(
    geo_data=stadtbezirke,
    data=stadtbezirke,
    columns=["bez", "anteil_weiblich"],
    key_on="feature.properties.bez",
    fill_color="PuRd",
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name="Anteil weiblich benannter Straßen",
).add_to(m)

# Tooltip für Bezirke
folium.GeoJson(
    stadtbezirke,
    tooltip=folium.GeoJsonTooltip(
        fields=["bez", "anteil_weiblich"],
        aliases=["Bezirk", "Anteil Frauen (%)"],
        localize=True,
        sticky=False
    ),
    style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.5}
).add_to(m)


# 7. Legende für Linienfarbe
legend_html = """
{% macro html(this, kwargs) %}
<div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 120px;
            z-index:9999; font-size:14px; background-color:white;
            border:2px solid grey; padding:10px;">
<b>Legende</b><br>
<span style="color:blue;">■</span> Männer<br>
<span style="color:red;">■</span> Frauen<br>
<span style="color:gray;">■</span> Unbekannt
</div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)


# 8. Straßen farblich einzeichnen
color_map = {"männlich": "blue", "weiblich": "red"}

folium.GeoJson(
    personen_strassen,
    style_function=lambda feature: {
        "color": color_map.get(feature["properties"]["Geschlecht"], "gray"),
        "weight": 2,
        "opacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["strasse", "Geschlecht"],
        aliases=["Straße", "Geschlecht"],
        localize=True,
        sticky=False
    )
).add_to(m)

# 9. Karte speichern
m.save("../../results/maps/Stadtbezirke/Verteilung_geschlecht_Stadtbezirke.html")

C:\Users\juliu\AppData\Local\Temp\ipykernel_9232\323278904.py:63: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadtbezirke.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_9232\323278904.py:63: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadtbezirke.geometry.centroid.unary_union.centroid


### PNG Stadtbezirke Geschlechterverteilung

In [4]:
# # 1. Daten laden & Geometrien bereinigen
def safe_from_wkt(x):
    """Sicheres Parsen von WKT → Shapely-Geometrie, sonst None."""
    if pd.isna(x):
        return None
    try:
        return from_wkt(str(x))
    except Exception:
        return None

# SRID entfernen + Geometrien parsen
df_streets["geometry"] = (
    df_streets["geom"]
    .astype(str)
    .str.replace(r"SRID=\d+;", "", regex=True)
    .apply(safe_from_wkt)
)

# Als GeoDataFrame mit gültigen Geometrien
df_streets = gpd.GeoDataFrame(df_streets, geometry="geometry", crs="EPSG:4326")
df_streets = df_streets.dropna(subset=["geometry"])


# Straßennamen vereinheitlichen
df_streets["strasse_clean"] = df_streets["strasse"].str.strip().str.lower()
df_persons["street_name_clean"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge Gender-Info
edges = df_streets.merge(
    df_persons[["street_name_clean", "Geschlecht"]],
    left_on="strasse_clean", right_on="street_name_clean", how="left"
)

# 2. Stadtbezirke laden
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L139/items"
try:
    stadtbezirke_json = requests.get(stadtbezirke_url).json()
    stadtbezirke = gpd.GeoDataFrame.from_features(stadtbezirke_json["features"])
    stadtbezirke = stadtbezirke.set_crs(epsg=4326)
except Exception as e:
    print(f"Fehler beim Laden der Stadtbezirke: {e}")


# 3. Räumliche Verknüpfung: Straße ↔ Bezirk
edges_mit_bezirk = gpd.sjoin(edges, stadtbezirke, how="left", predicate="intersects")

# 4. Nur nach Personen benannte Straßen + Berechnung
personen_strassen = edges_mit_bezirk.dropna(subset=["Geschlecht"])
personen_strassen_einzigartig = personen_strassen.drop_duplicates(subset=["bez", "strasse_clean"])

# Anteil weiblich je Bezirk
gesamt = personen_strassen_einzigartig.groupby("bez").size()
weiblich = (
    personen_strassen_einzigartig[personen_strassen_einzigartig["Geschlecht"] == "weiblich"]
    .groupby("bez")
    .size()
)
anteile = (weiblich / gesamt).fillna(0).reset_index()
anteile.columns = ["bez", "anteil_weiblich"]

# Merge mit GeoDataFrame der Stadtbezirke
stadt_gdf_gender = stadtbezirke.merge(anteile, on="bez", how="left")
stadt_gdf_gender["anteil_weiblich"] = stadt_gdf_gender["anteil_weiblich"].fillna(0)

# Spalte für die Legende (Werte von 0-100)
stadt_gdf_gender["anteil_prozent"] = stadt_gdf_gender["anteil_weiblich"] * 100


# STATISCHE PNG-KARTE ERSTELLEN

target_crs = "EPSG:3857"

# 5. Daten für Plotting vorbereiten (CRS umwandeln)
stadt_plot = stadt_gdf_gender.to_crs(target_crs)
personen_strassen_plot = personen_strassen.to_crs(target_crs)

# 6. Karte erstellen
fig, ax = plt.subplots(figsize=(12, 12))

# A: Bezirke als Choropleth plotten (Anteil Frauen)
stadt_plot.plot(
    ax=ax,
    column="anteil_prozent",
    cmap="PuRd", 
    legend=True,
    alpha=0.6,
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={
        'label': "Anteil Frauen (%)",
        'shrink': 0.75,
        'aspect': 30
    }
)

# Legenden-Schriftgröße anpassen
cax = fig.axes[-1]
cax.tick_params(labelsize=8)
cax.yaxis.label.set_size(9)

# B: Straßen-Linien plotten (Farbe nach Geschlecht)
strassen_weiblich = personen_strassen_plot[personen_strassen_plot["Geschlecht"] == "weiblich"]
strassen_maennlich = personen_strassen_plot[personen_strassen_plot["Geschlecht"] == "männlich"]
strassen_unbekannt = personen_strassen_plot.loc[personen_strassen_plot["Geschlecht"].isna() | 
                                               (personen_strassen_plot["Geschlecht"] == "unbekannt")]

# Straßen nach Geschlecht plotten

# Weiblich
strassen_weiblich.plot(ax=ax, color="red", linewidth=1.2, alpha=0.8, label="Frauen (Straße)")
# Männlich
strassen_maennlich.plot(ax=ax, color="blue", linewidth=1.2, alpha=0.8, label="Männer (Straße)")
# Unbekannt/Nicht benannt nach Person
# strassen_unbekannt.plot(ax=ax, color="gray", linewidth=0.7, alpha=0.5, label="Andere/Unbekannt (Straße)")


# Manuelle Legende für die Linien-Plots hinzufügen
ax.legend(loc='lower left', frameon=True, fontsize=10)


# C: Hintergrundkarte hinzufügen
ctx.add_basemap(
    ax,
    crs=target_crs,
    source=ctx.providers.OpenStreetMap.Mapnik,
    zoom=12
)


# 7. Karte finalisieren & als PNG speichern
ax.set_axis_off()
plt.title("Anteil nach Frauen benannter Straßen pro Stadtbezirk", fontsize=16)

filename = "../../results/images/Stadtbezirke/Karte_Stadtbezirk_Geschlecht_Frauenanteil.png"

plt.savefig(
    filename,
    dpi=300,
    bbox_inches="tight"
)
plt.close(fig)

### Stadtbezirk Vergleich

In [19]:
# 1. Gesamtanzahl nach Geschlecht pro Bezirk
geschlecht_counts = personen_strassen_einzigartig.groupby(["bez", "Geschlecht"]).size().unstack(fill_value=0)

# 2. Optional: "Unbekannt" hinzufügen (Straßen ohne Geschlecht)
# Zuerst alle Bezirke
alle_bezirke = stadtbezirke["bez"].unique()
# Dann sicherstellen, dass alle Geschlechter vorhanden sind
for g in ["weiblich", "männlich", "trans* Frau"]:
    if g not in geschlecht_counts.columns:
        geschlecht_counts[g] = 0

# 3. Anteile berechnen
geschlecht_anteile = geschlecht_counts.div(geschlecht_counts.sum(axis=1), axis=0)
# 3.1 Daten sortieren
geschlecht_anteile_sortiert = geschlecht_anteile.sort_values(by="weiblich", ascending=True)

# 4. Balkendiagramm erstellen
fig, ax = plt.subplots(figsize=(10,6))

geschlecht_anteile_sortiert.plot(kind="barh", stacked=True, color=["blue", "gray", "red"], ax=ax)

# 4.1 X-Achse in Prozent
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

ax.set_ylabel("Stadtbezirk")
ax.set_xlabel("Anteil der Straßen")
ax.set_title("Anteil nach Männer/Frauen benannte Straßen pro Bezirk")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Geschlecht")
plt.tight_layout()

# 5. Diagramm speichern
plt.savefig("../../results/charts/bar_charts_district_comparison/Übersicht_Verteilung_Geschlecht_Stadtbezirke.png", dpi=300)
plt.close()

## Karte Gemarkungen

In [ ]:
# 1. Daten laden & Geometrien bereinigen

def safe_from_wkt(x):
    """Sicheres Parsen von WKT → Shapely-Geometrie, sonst None."""
    if pd.isna(x):
        return None
    try:
        return from_wkt(str(x))
    except Exception:
        return None


# SRID entfernen + Geometrien parsen
df_streets["geometry"] = (
    df_streets["geom"]
    .astype(str)
    .str.replace(r"SRID=\d+;", "", regex=True)
    .apply(safe_from_wkt)
)

# Als GeoDataFrame mit gültigen Geometrien
df_streets = gpd.GeoDataFrame(df_streets, geometry="geometry", crs="EPSG:4326")
df_streets = df_streets.dropna(subset=["geometry"])


# Straßennamen vereinheitlichen
df_streets["strasse_clean"] = df_streets["strasse"].str.strip().str.lower()
df_persons["street_name_clean"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge Gender-Info
edges = df_streets.merge(
    df_persons[["street_name_clean", "Geschlecht"]],
    left_on="strasse_clean", right_on="street_name_clean", how="left"
)

# 3. Stadtbezirke laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
stadtbezirke_json = requests.get(stadtbezirke_url).json()
stadtbezirke = gpd.GeoDataFrame.from_features(stadtbezirke_json["features"])
stadtbezirke = stadtbezirke.set_crs(epsg=4326)

# 4. Räumliche Verknüpfung: Straße ↔ Gemarkung
edges_mit_bezirk = gpd.sjoin(edges, stadtbezirke, how="left", predicate="intersects")

# 5. NUR nach Personen benannte Straßen
personen_strassen = edges_mit_bezirk.dropna(subset=["Geschlecht"])
personen_strassen_einzigartig = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_clean"])

# Anteil weiblich je Gemarkung
gesamt = personen_strassen_einzigartig.groupby("gem_nam").size()
weiblich = (
    personen_strassen_einzigartig[personen_strassen_einzigartig["Geschlecht"] == "weiblich"]
    .groupby("gem_nam")
    .size()
)
anteile = (weiblich / gesamt).fillna(0).reset_index()
anteile.columns = ["gem_nam", "anteil_weiblich"]

stadtbezirke = stadtbezirke.merge(anteile, on="gem_nam", how="left")
stadtbezirke["anteil_weiblich"] = stadtbezirke["anteil_weiblich"].fillna(0)

# 6. Karte erstellen
center = stadtbezirke.geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=12)

# Choropleth
folium.Choropleth(
    geo_data=stadtbezirke,
    data=stadtbezirke,
    columns=["gem_nam", "anteil_weiblich"],
    key_on="feature.properties.gem_nam",
    fill_color="PuRd",
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name="Anteil weiblich benannter Straßen",
).add_to(m)

# Tooltip für Bezirke
folium.GeoJson(
    stadtbezirke,
    tooltip=folium.GeoJsonTooltip(
        fields=["gem_nam", "anteil_weiblich"],
        aliases=["Bezirk", "Anteil Frauen (%)"],
        localize=True,
        sticky=False
    ),
    style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.5}
).add_to(m)


# 7. Legende für Linienfarbe
legend_html = """
{% macro html(this, kwargs) %}
<div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 120px;
            z-index:9999; font-size:14px; background-color:white;
            border:2px solid grey; padding:10px;">
<b>Legende</b><br>
<span style="color:blue;">■</span> Männer<br>
<span style="color:red;">■</span> Frauen<br>
<span style="color:gray;">■</span> Unbekannt
</div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)


# 8. Straßen farblich einzeichnen (GeoJson statt Schleife)
color_map = {"männlich": "blue", "weiblich": "red"}

folium.GeoJson(
    personen_strassen,
    style_function=lambda feature: {
        "color": color_map.get(feature["properties"]["Geschlecht"], "gray"),
        "weight": 2,
        "opacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["strasse", "Geschlecht"],
        aliases=["Straße", "Geschlecht"],
        localize=True,
        sticky=False
    )
).add_to(m)

# 9. Karte speichern
m.save("../../results/maps/Gemarkungen/Verteilung_geschlecht_Gemarkung.html")


C:\Users\juliu\AppData\Local\Temp\ipykernel_9232\594819762.py:63: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadtbezirke.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_9232\594819762.py:63: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadtbezirke.geometry.centroid.unary_union.centroid


### PNG Karte Geschlecht Gemarkungen

In [ ]:
# # 1. Daten laden & Geometrien bereinigen
def safe_from_wkt(x):
    """Sicheres Parsen von WKT → Shapely-Geometrie, sonst None."""
    if pd.isna(x):
        return None
    try:
        return from_wkt(str(x))
    except Exception:
        return None

# SRID entfernen + Geometrien parsen
df_streets["geometry"] = (
    df_streets["geom"]
    .astype(str)
    .str.replace(r"SRID=\d+;", "", regex=True)
    .apply(safe_from_wkt)
)

# Als GeoDataFrame mit gültigen Geometrien
df_streets = gpd.GeoDataFrame(df_streets, geometry="geometry", crs="EPSG:4326")
df_streets = df_streets.dropna(subset=["geometry"])


# Straßennamen vereinheitlichen
df_streets["strasse_clean"] = df_streets["strasse"].str.strip().str.lower()
df_persons["street_name_clean"] = df_persons["Straßenname"].str.strip().str.lower()

# Merge Gender-Info
edges = df_streets.merge(
    df_persons[["street_name_clean", "Geschlecht"]],
    left_on="strasse_clean", right_on="street_name_clean", how="left"
)

# 2. Gemarkungen laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadtbezirke_json = requests.get(stadtbezirke_url).json()
    stadtbezirke = gpd.GeoDataFrame.from_features(stadtbezirke_json["features"])
    stadtbezirke = stadtbezirke.set_crs(epsg=4326)
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")


# 3. Räumliche Verknüpfung: Straße ↔ Bezirk
edges_mit_bezirk = gpd.sjoin(edges, stadtbezirke, how="left", predicate="intersects")

# 4. Nur nach Personen benannte Straßen + Berechnung
personen_strassen = edges_mit_bezirk.dropna(subset=["Geschlecht"])
personen_strassen_einzigartig = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_clean"])

# Anteil weiblich je Bezirk
gesamt = personen_strassen_einzigartig.groupby("gem_nam").size()
weiblich = (
    personen_strassen_einzigartig[personen_strassen_einzigartig["Geschlecht"] == "weiblich"]
    .groupby("gem_nam")
    .size()
)
anteile = (weiblich / gesamt).fillna(0).reset_index()
anteile.columns = ["gem_nam", "anteil_weiblich"]

# Merge mit GeoDataFrame der Gemarkungen
stadt_gdf_gender = stadtbezirke.merge(anteile, on="gem_nam", how="left")
stadt_gdf_gender["anteil_weiblich"] = stadt_gdf_gender["anteil_weiblich"].fillna(0)

# Spalte für die Legende (Werte von 0-100)
stadt_gdf_gender["anteil_prozent"] = stadt_gdf_gender["anteil_weiblich"] * 100


#STATISCHE PNG-KARTE ERSTELLEN

target_crs = "EPSG:3857"

# 5. Daten für Plotting vorbereiten (CRS umwandeln)
stadt_plot = stadt_gdf_gender.to_crs(target_crs)
personen_strassen_plot = personen_strassen.to_crs(target_crs)

# 6. Karte erstellen
fig, ax = plt.subplots(figsize=(12, 12))

# A: Bezirke als Choropleth plotten (Anteil Frauen)
stadt_plot.plot(
    ax=ax,
    column="anteil_prozent",
    cmap="PuRd", 
    legend=True,
    alpha=0.6,
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={
        'label': "Anteil Frauen (%)",
        'shrink': 0.75,
        'aspect': 30
    }
)

# Legenden-Schriftgröße anpassen
cax = fig.axes[-1]
cax.tick_params(labelsize=8)
cax.yaxis.label.set_size(9)

# B: Straßen-Linien plotten (Farbe nach Geschlecht)
strassen_weiblich = personen_strassen_plot[personen_strassen_plot["Geschlecht"] == "weiblich"]
strassen_maennlich = personen_strassen_plot[personen_strassen_plot["Geschlecht"] == "männlich"]
strassen_unbekannt = personen_strassen_plot.loc[personen_strassen_plot["Geschlecht"].isna() | 
                                               (personen_strassen_plot["Geschlecht"] == "unbekannt")]

# Straßen nach Geschlecht plotten (für die Linien-Legende)en.

# Weiblich
strassen_weiblich.plot(ax=ax, color="red", linewidth=1.2, alpha=0.8, label="Frauen (Straße)")
# Männlich
strassen_maennlich.plot(ax=ax, color="blue", linewidth=1.2, alpha=0.8, label="Männer (Straße)")


# Manuelle Legende für die Linien-Plots hinzufügen
ax.legend(loc='lower left', frameon=True, fontsize=10)


# C: Hintergrundkarte hinzufügen
ctx.add_basemap(
    ax,
    crs=target_crs,
    source=ctx.providers.OpenStreetMap.Mapnik,
    zoom=12
)


# 7. Karte finalisieren & als PNG speichern
ax.set_axis_off()
plt.title("Anteil nach Frauen benannter Straßen pro Gemarkung", fontsize=16)

filename = "../../results/images/Gemarkungen/Karte_Gemarkung_Geschlecht_Frauenanteil.png"

plt.savefig(
    filename,
    dpi=300,
    bbox_inches="tight"
)
plt.close(fig)

### Gemarkungen Vergleich

In [ ]:
# 1. Gesamtanzahl nach Geschlecht pro Gemarkung
geschlecht_counts = personen_strassen_einzigartig.groupby(["gem_nam", "Geschlecht"]).size().unstack(fill_value=0)

# 2. Optional: "Unbekannt" hinzufügen (Straßen ohne Geschlecht)
# Zuerst alle Gemarkung
alle_bezirke = stadtbezirke["gem_nam"].unique()
# Dann sicherstellen, dass alle Geschlechter vorhanden sind
for g in ["weiblich", "männlich", "trans* Frau"]:
    if g not in geschlecht_counts.columns:
        geschlecht_counts[g] = 0

# 3. Anteile berechnen
geschlecht_anteile = geschlecht_counts.div(geschlecht_counts.sum(axis=1), axis=0)
# 3.1 Daten sortieren
geschlecht_anteile_sortiert = geschlecht_anteile.sort_values(by="weiblich", ascending=True)

# Berechnung der optimalen Höhe basierend auf der Anzahl der Bezirke
num_bezirke = len(geschlecht_anteile_sortiert)
# Schätzung: 0.5 Zoll pro Bezirk für eine lesbare Beschriftung
fig_height = max(6, num_bezirke * 0.5)

# 4. Balkendiagramm erstellen
fig, ax = plt.subplots(figsize=(10, fig_height)) 

# Anmerkung: Stellen Sie sicher, dass die Farben ("blue", "gray", "red") zur Spaltenreihenfolge 
geschlecht_anteile_sortiert.plot(kind="barh", stacked=True, color=["blue", "gray", "red"], ax=ax)

# 4.1 X-Achse in Prozent
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

ax.set_ylabel("Gemarkung") 
ax.set_xlabel("Anteil der Straßen")
ax.set_title("Anteil nach Männer/Frauen benannte Straßen pro Gemarkung")

plt.legend(title="Geschlecht")
plt.tight_layout()

# 5. Diagramm speichern
plt.savefig("../../results/charts/bar_charts_district_comparison/Übersicht_Verteilung_Geschlecht_Gemarkungen.png", dpi=300)
plt.close()